In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader


# Load MNIST DATA

In [2]:
transform = transforms.Compose([
    transforms.ToTensor(),  # Converts to tensor [0, 1]
    transforms.Normalize((0.1307,), (0.3081,))  # Mean, Std for MNIST
])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)


100.0%


# Define a CNN architecture

In [3]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)  # (28x28) → (28x28)
        self.pool = nn.MaxPool2d(2, 2)               # (28x28) → (14x14)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1) # (14x14) → (14x14)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)         # Flatten → FC
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))         # (28x28) → (14x14)
        x = self.pool(F.relu(self.conv2(x)))         # (14x14) → (7x7)
        x = x.view(-1, 64 * 7 * 7)                    # Flatten
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


# Train the model

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNN().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

epochs = 5
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        output = model(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    print(f"Epoch [{epoch+1}/{epochs}] - Loss: {total_loss:.4f}")


Epoch [1/5] - Loss: 126.2846
Epoch [2/5] - Loss: 40.2962
Epoch [3/5] - Loss: 26.4661
Epoch [4/5] - Loss: 20.6739
Epoch [5/5] - Loss: 14.3947


# Evaluate on test set

In [5]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        output = model(images)
        _, preds = torch.max(output.data, 1)
        total += labels.size(0)
        correct += (preds == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")


Test Accuracy: 98.97%


# Save the model

In [6]:
torch.save(model.state_dict(), "mnist_cnn.pth")
